# 06 - Model Deployment

The purpose of this notebook is to execute a CI/CD routine to test and deploy the trained model to `Vertex AI` as an `Endpoint` for online prediction serving. The notebook covers the following steps:
1. Run the test steps locally.
2. Execute the model deployment `CI/CD` steps using `Cloud Build`.



## Setup

### Import libraries

In [1]:
import os
import logging

logging.getLogger().setLevel(logging.INFO)

### Setup Google Cloud project

In [2]:
PROJECT = 'mwpmltr' # Change to your project id.
REGION = 'us-central1' # Change to your region.

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]

print("Project ID:", PROJECT)
print("Region:", REGION)

Project ID: mwpmltr
Region: us-central1


### Set configurations

In [3]:
VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
ENDPOINT_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

LD_LIBRARY_PATH = f'/opt/conda/lib/'

## 1. Run CI/CD steps locally

In [4]:
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['MODEL_DISPLAY_NAME'] = MODEL_DISPLAY_NAME
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME
os.environ["LD_LIBRARY_PATH"] = LD_LIBRARY_PATH

### Run the model artifact testing

In [5]:
!pytest src/tests/model_deployment_tests.py::test_model_artifact \
    -s \
    --disable-warnings \
    --log-cli-level INFO

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.6.1
collecting ... 2022-09-11 00:09:25.541134: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/
2022-09-11 00:09:25.541190: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
collected 1 item                                                               

src/tests/model_deployment_tests.py::test_model_artifact 2022-09-11 00:09:29.845771: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory

### Run create endpoint

In [6]:
!python build/utils.py \
    --mode=create-endpoint\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}

### Run deploy model

In [7]:
!python build/utils.py \
    --mode=deploy-model\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

Deploying Model projects/55590906972/locations/us-central1/models/5832755253723791360 to Endpoint : projects/55590906972/locations/us-central1/endpoints/3874177598980358144
INFO:google.cloud.aiplatform.models:Deploying Model projects/55590906972/locations/us-central1/models/5832755253723791360 to Endpoint : projects/55590906972/locations/us-central1/endpoints/3874177598980358144
/opt/conda/lib/python3.7/site-packages/proto/marshal/rules/enums.py:40: UserWarning: Unrecognized DeploymentResourcesType enum value: 3
  value=value,
Deploy Endpoint model backing LRO: projects/55590906972/locations/us-central1/endpoints/3874177598980358144/operations/2695294769415847936
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/55590906972/locations/us-central1/endpoints/3874177598980358144/operations/2695294769415847936
Endpoint model deployed. Resource name: projects/55590906972/locations/us-central1/endpoints/3874177598980358144
INFO:google.cloud.aiplatform.models:Endp

### Test deployed model endpoint

In [8]:
!pytest src/tests/model_deployment_tests.py::test_model_endpoint --disable-warnings

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.6.1
collected 1 item                                                               

src/tests/model_deployment_tests.py .                                    [100%]

======================== 1 passed, 10 warnings in 5.32s ========================


## 2. Execute the Model Deployment CI/CD routine in Cloud Build

The CI/CD routine is defined in the [model-deployment.yaml](model-deployment.yaml) file, and consists of the following steps:
1. Load and test the the trained model interface.
2. Create and endpoint in Vertex AI if it doesn't exists.
3. Deploy the model to the endpoint.
4. Test the endpoint.

### Build CI/CD container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying model will be executed.

In [9]:
!echo $CICD_IMAGE_URI

gcr.io/mwpmltr/cicd:latest


In [10]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

Creating temporary tarball archive of 10 file(s) totalling 22.5 KiB before compression.
Uploading tarball of [build/.] to [gs://mwpmltr_cloudbuild/source/1662855841.411134-61b9b07a09f146fcb121c395534f4505.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mwpmltr/locations/global/builds/57ae59aa-a134-45e3-a9cb-80d471da6236].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/57ae59aa-a134-45e3-a9cb-80d471da6236?project=55590906972].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "57ae59aa-a134-45e3-a9cb-80d471da6236"

FETCHSOURCE
Fetching storage object: gs://mwpmltr_cloudbuild/source/1662855841.411134-61b9b07a09f146fcb121c395534f4505.tgz#1662855841725131
Copying gs://mwpmltr_cloudbuild/source/1662855841.411134-61b9b07a09f146fcb121c395534f4505.tgz#1662855841725131...
/ [1 files][  3.4 KiB/  3.4 KiB]                                                
Operation completed over 1 objects/3.4 KiB.
BUILD
Already h

### Run CI/CD from model deployment using Cloud Build

In [11]:
REPO_URL = "https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai.git" # Change to your github repo.
BRANCH = "main" 

In [12]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
"""

!echo $SUBSTITUTIONS

_REPO_URL=https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai.git,_BRANCH=main,_CICD_IMAGE_URI=gcr.io/mwpmltr/cicd:latest,_PROJECT=mwpmltr,_REGION=us-central1,_MODEL_DISPLAY_NAME=chicago-taxi-tips-classifier-v01,_ENDPOINT_DISPLAY_NAME=chicago-taxi-tips-classifier,


In [13]:
!gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --timeout=30m

Created [https://cloudbuild.googleapis.com/v1/projects/mwpmltr/locations/global/builds/04af1b40-003c-41dc-ac62-eea551117406].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/04af1b40-003c-41dc-ac62-eea551117406?project=55590906972].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "04af1b40-003c-41dc-ac62-eea551117406"

FETCHSOURCE
BUILD
Starting Step #0 - "Clone Repository"
Step #0 - "Clone Repository": Already have image (with digest): gcr.io/cloud-builders/git
Step #0 - "Clone Repository": Cloning into 'mlops-with-vertex-ai'...
Step #0 - "Clone Repository": POST git-upload-pack (352 bytes)
Step #0 - "Clone Repository": POST git-upload-pack (194 bytes)
Finished Step #0 - "Clone Repository"
Starting Step #1 - "Test Model Artifact"
Step #1 - "Test Model Artifact": Pulling image: gcr.io/mwpmltr/cicd:latest
Step #1 - "Test Model Artifact": latest: Pulling from mwpmltr/cicd
Step #1 - "Test Model Artifact": d5fd17ec1